<a href="https://colab.research.google.com/github/AabidMK/Speech-to-Image-Live-Conversion-using-Deep-Learning_Infosys_Internship_Oct2024/blob/Neeharika/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-pflq5kph
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-pflq5kph
  Resolved https://github.com/openai/whisper.git to commit 271445b2f24f00f8175c4fb7ae91876f7451dfc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803557 sha256=c9ec73e390b722c66bec51cf71abf578599cdade0be985e9a0c73a5224555f0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-r3pobxl1/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [2]:
!pip install openai

In [3]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.2 MB/s eta 0:00:00


In [4]:
!pip install torch

In [5]:
!pip install openai-whisper  # Install the whisper package
import whisper  # Import the whisper module


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import whisper
from jiwer import wer

csv_path ="/content/drive/MyDrive/sample_dataset/filtered_csv_file.csv"
audio_folder_path ="/content/drive/MyDrive/sample_dataset/train"

df= pd.read_csv(csv_path)
df["file_name"]= df["file_name"].apply(lambda x: os.path.join(audio_folder_path, os.path.basename(x)))
df.to_csv(csv_path, index=False)

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/sample_dataset/filtered_csv_file.csv")

dataset = dataset.cast_column("file_name", Audio(sampling_rate=16000))
dataset = dataset.rename_column("file_name", "audio")
dataset = dataset.rename_column('phrase','sentence')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import re
from datasets import load_dataset, Audio
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base")

def prepare_dataset(batch):
    # Process audio to input features
    batch["input_features"] = processor(batch["audio"]["array"], sampling_rate=16000).input_features[0]
    # Process transcription to labels
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

# Specify only the columns you need to keep
columns_to_remove = [
    'audio_clipping', 'audio_clipping:confidence', 'background_noise_audible',
    'background_noise_audible:confidence', 'overall_quality_of_the_audio',
    'quiet_speaker', 'quiet_speaker:confidence', 'speaker_id', 'file_download',
    'prompt', 'writer_id'
]

# Load the dataset
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/sample_dataset/filtered_csv_file.csv')

# Function to convert Windows paths to Colab paths
def convert_path(path):
    # Extract filename from the Windows path using regex
    filename = re.search(r"[^\\/]+$", path).group(0)
    # Construct the correct Colab path
    colab_path = f"/content/drive/MyDrive/sample_dataset/train/{filename}"  # Update 'train' if your audio files are in a different folder
    return colab_path


dataset = dataset.map(lambda example: {"file_name": convert_path(example["file_name"])})
dataset = dataset.cast_column('file_name', Audio(sampling_rate=16000))

dataset = dataset.rename_column('file_name', 'audio')
dataset = dataset.rename_column('phrase', 'sentence')

# Apply the processing function to the dataset
dataset = dataset.map(prepare_dataset, remove_columns=columns_to_remove)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.nn.utils.rnn import pad_sequence


model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")


training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=3,
    fp16=False,
    save_steps=100,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
)

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
class DataCollatorForWhisper:
    def __call__(self, features):

        input_features = [torch.tensor(feature["input_features"]) for feature in features]
        labels = [torch.tensor(feature["labels"]) for feature in features]


        input_features_padded = pad_sequence(input_features, batch_first=True, padding_value=0)
        labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)

        return {
            "input_features": input_features_padded,
            "labels": labels_padded
        }

In [ ]:
data_collator = DataCollatorForWhisper()

In [ ]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    run_name='my_unique_run_name',  # Change this to a unique name
    # other arguments...
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
model.save_pretrained("whisper-finetuned")
processor.save_pretrained("whisper-finetuned")

In [8]:
pip install streamlit_jupyter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59

In [12]:
!pip install sounddevice

In [28]:
!pip install sounddevice
!pip install streamlit
!pip install whisper
!pip install torch
!pip install diffusers
!pip install scipy


import streamlit as st
import whisper
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import sounddevice as sd # Import the sounddevice library after installing it.
from scipy.io.wavfile import write
import numpy as np

In [2]:
!apt-get update
!apt-get install libportaudio2

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [59.5 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [3]:
import streamlit as st
import whisper
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np

# Load Whisper and Stable Diffusion models
@st.cache_resource  # Cache models to prevent reloading each time
def load_models():
    whisper_model = whisper.load_model("base")  # Load Whisper model for transcription
    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")  # Load Stable Diffusion model
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    return whisper_model, pipe

whisper_model, pipe = load_models()

# Streamlit UI
st.title("Audio-to-Image Generator with Whisper & Stable Diffusion")

st.write("Record an audio input and generate an image based on the transcribed text.")

# Define recording duration
duration = st.slider("Select duration of recording (seconds):", 1, 10, 3)

# Record and transcribe when the "Record" button is clicked
if st.button("Record"):
    st.write("Recording...")
    fs = 44100  # Sample rate for audio recording
    audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # Wait until recording finishes
    st.write("Recording complete!")

    # Save the recorded audio to a .wav file
    audio_path = "recorded_audio.wav"
    write(audio_path, fs, audio_data)

    # Step 1: Transcribe audio with Whisper
    st.write("Transcribing audio with Whisper model...")
    transcription = whisper_model.transcribe(audio_path)
    st.write(f"Transcription: {transcription['text']}")

    # Step 2: Generate image based on transcription
    st.write("Generating image with Stable Diffusion...")
    generated_image = pipe(transcription["text"]).images[0]

    # Display the generated image
    st.image(generated_image, caption="Generated Image")

2024-11-05 11:30:53.615 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 11:30:53.709 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-05 11:30:53.710 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 11:30:53.713 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
 24%|█████████                             | 33.0M/139M [00:00<00:01, 94.2MiB/s]2024-11-05 11:30:54.217 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 11:30:54.219 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 83.6MiB/s]
/usr/local/lib/p

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-11-05 11:31:43.305 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 11:31:43.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 11:31:43.308 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 11:31:43.309 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 11:31:43.311 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when ru

In [4]:
! pip install streamlit -q

In [5]:
!wget -q -O - ipv4.icanhazip.com

107.167.185.20


In [6]:
%%writefile app.py

Writing app.py


In [10]:
!npm install -g localtunnel@2.0.2


added 22 packages, and audited 23 packages in 4s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [32]:
# Install necessary libraries
!pip install git+https://github.com/openai/whisper.git
!pip install SpeechRecognition requests Pillow

# Import necessary libraries
import whisper
import speech_recognition as sr
from IPython.display import Javascript, HTML, display
from google.colab import output
import base64
import requests
from PIL import Image
from io import BytesIO

# Load the Whisper model
model = whisper.load_model("base")  # You can choose "tiny", "base", "small", "medium", or "large"

# JavaScript code for recording audio
RECORD_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
var record = function() {
    const my_recorder = new AudioRecorder();
    my_recorder.start();
    console.log("Recording...");
    return sleep(3000).then(() => {
        my_recorder.stop();
        console.log("Stopped Recording");
        return my_recorder.getAudioBlob();
    });
};

class AudioRecorder {
    constructor() {
        this.audioContext = new (window.AudioContext || window.webkitAudioContext)();
        this.stream = null;
        this.recorder = null;
        this.audioChunks = [];
    }

    async start() {
        this.stream = await navigator.mediaDevices.getUser Media({ audio: true });
        this.recorder = new MediaRecorder(this.stream);
        this.recorder.ondataavailable = event => this.audioChunks.push(event.data);
        this.recorder.start();
    }

    stop() {
        return new Promise(resolve => {
            this.recorder.onstop = () => {
                const audioBlob = new Blob(this.audioChunks, { type: 'audio/wav' });
                resolve(audioBlob);
            };
            this.recorder.stop();
        });
    }

    getAudioBlob() {
        return this.stop().then(audioBlob => {
            let reader = new FileReader();
            reader.readAsDataURL(audioBlob);
            reader.onloadend = () => {
                google.colab.kernel.invokeFunction('notebook.createAudio', [reader.result], {});
            };
        });
    }
};

record();
"""

# Register a callback to save the audio file
def create_audio(audio_base64):
    audio_data = audio_base64.split(',')[1]
    audio_file_path = "recorded_audio.wav"
    with open(audio_file_path, "wb") as audio_file:
        audio_file.write(base64.b64decode(audio_data))
    return audio_file_path

output.register_callback('notebook.createAudio', create_audio)

# Display the JavaScript interface for recording
display(HTML('<button onclick="record()">Record Audio</button>'))
display(Javascript(RECORD_JS))

# Function to transcribe audio to text using Whisper
def transcribe_audio(audio_file_path):
    # Load audio and transcribe using Whisper
    result = model.transcribe(audio_file_path)
    return result['text']

# After recording and transcribing the audio, use the functions
# Note: The following lines should be executed after the audio is recorded
# Uncomment the lines below after recording the audio
# audio_file_path = "recorded_audio.wav"  # Path to the recorded audio file
# transcribed_text = transcribe_audio(audio_file_path)
# print(f"Transcribed Text: {transcribed_text}")

# Generate image from the transcribed text
# Uncomment and replace with your image generation code as needed
# generate_image_from_text(transcribed_text)

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-kn192peu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-kn192peu
  Resolved https://github.com/openai/whisper.git to commit 271445b2f24f00f8175c4fb7ae91876f7451dfc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


<IPython.core.display.Javascript object>

In [34]:
# Record audio
if st.button("Record Audio"):
    duration = st.slider("Select recording duration in seconds", 1, 10, 5)
    audio = record_audio(duration)
    text = transcribe_audio(audio)
    st.write(f"Recognized: {text}")
    # Load the stable diffusion model
    model_id = "CompVis/stable-diffusion-v1-4"
    pipe = StableDiffusionPipeline.from_pretrained(model_id)
    with torch.autocast("cuda"):
        image = pipe(text, guidance_scale=7.5).images[0]

    # Display the image
    st.image(image, caption="Generated Image", use_column_width=True)

2024-11-05 12:26:12.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 12:26:12.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 12:26:12.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 12:26:12.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 12:26:12.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [35]:
pip install streamlit whisper sounddevice torch diffusers

In [ ]:
# Install necessary libraries
!pip install streamlit whisper sounddevice torch diffusers

# Create a Streamlit app
app_code = """
import streamlit as st
import whisper
import sounddevice as sd
import numpy as np
import torch
from diffusers import StableDiffusionPipeline

# Load the Whisper model
model = whisper.load_model("base")

# Function to record audio
def record_audio(duration=5, samplerate=16000):
    st.write("Recording...")
    audio = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1, dtype='float32')
    sd.wait()
    st.write("Recording finished.")
    return audio.flatten()

# Function to transcribe audio using Whisper
def transcribe_audio(audio):
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions(language="en")
    result = model.decode(mel, options)
    return result.text

# Streamlit app
def main():
    st.title("Audio Transcription and Image Generation")

    # Record audio
    if st.button("Record Audio"):
        duration = st.slider("Select recording duration in seconds", 1, 10, 5)
        audio = record_audio(duration)
        text = transcribe_audio(audio)
        st.write(f"Recognized: {text}")

        # Load the stable diffusion model
        model_id = "CompVis/stable-diffusion-v1-4"
        pipe = StableDiffusionPipeline.from_pretrained(model_id)
        with torch.autocast("cuda"):
            image = pipe(text, guidance_scale=7.5).images[0]

        # Display the image
        st.image(image, caption="Generated Image", use_column_width=True)

if __name__ == "__main__":
    main()
"""

# Save the app code to a file
with open('app.py', 'w') as f:
    f.write(app_code)

# Run the Streamlit app
!streamlit run app.py &

# To access the app, you would typically need to use a tool like ngrok to expose the local server.
# However, Colab does not support this directly. You can run this code on your local machine instead.




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://107.167.185.20:8501

